# Input Data Pipeline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/Research/FIRE/2020-Speech-Recognition/amazing_grace/
%ls

In [ ]:
!pip install librosa
!pip install python_speech_features
!pip install lyrics-extractor

In [4]:
import tensorflow as tf
import keras
import librosa
import numpy as np
import os
import re
from lyrics_extractor import SongLyrics

In [5]:
%cd ..
%ls

/content/drive/My Drive/Research/FIRE/2020-Speech-Recognition
amazing_grace/        checker.py            keys.txt
amazing_grace.m4a     DataAugmentation.py   PinkPanther30.wav
amazing_grace.midi    data_checker.py       __pycache__/
amazing_grace.tar.gz  DataGenerator.py      README.md
amazing_grace.tsv     generator_test.ipynb  United_States_Top_50@


In [14]:
keys = open("keys.txt").readlines()
api_key = keys[0]
engine_id = keys[1]
extract_lyrics = SongLyrics(api_key, engine_id)

In [16]:
def process_lyrics(lyrics):
  lyrics = re.sub("[\(\[].*?[\)\]]", "", lyrics)
  lyrics = lyrics.replace('\n', ' ')
  return lyrics.lower()

lyrics = extract_lyrics.get_lyrics("Amazing Grace")
lyrics = process_lyrics(data['lyrics'])

Here, we load in our dataset using our generator.

In [17]:
import random
from os import listdir
from os.path import isfile, join
from DataGenerator import DataGenerator

batch_size = 50
AUTOTUNE = tf.data.experimental.AUTOTUNE

file_list = [f for f in listdir("amazing_grace/") if isfile(join("amazing_grace/", f))]
gen_args = ["amazing_grace/",file_list,[lyrics]*len(file_list),batch_size,(1,len(file_list)), 1,1,True]
batch = tf.data.Dataset.from_generator(DataGenerator, args=gen_args,
                                         output_types=tf.int32, 
                                         output_shapes = (batch_size, None, 80),)

batch = batch.prefetch(buffer_size=AUTOTUNE)
options = tf.data.Options()
options.experimental_threading.max_intra_op_parallelism = 1
batch = batch.with_options(options)
print(batch)

<_OptionsDataset shapes: (50, None, 80), types: tf.int32>
